## MO444 Project 3 - Reinforcement Learning

* Rodrigo Araújo Marinho Franco - RA: 233569
* Felipe Marinho Tavares - RA: 265680

### Contributions:
* Rodrigo
 * ...
* Felipe
 * ...

# PART I - Evolutionary Model

# PART II - Reinforcement Learning

In [1]:
from util import *
import numpy as np
import matplotlib.pyplot as plt

import pacman as pm
from pacman import Directions
import layout as l
import textDisplay
import dqnAgent, ghostAgents

try:
    import boinc
    _BOINC_ENABLED = True
except:
    _BOINC_ENABLED = False

In [2]:
class Environment:
    def __init__(self, layout="mediumClassic"):
        self.layout = l.getLayout(layout)
        state_size = self.layout.height*self.layout.width
        self.beQuiet=True
        self.catchExceptions = False
        self.rules = pm.ClassicGameRules(timeout=30)
        self.pacman = dqnAgent.DQNAgent(state_size, action_size=5, seed=27)
        self.reset()

    def reset(self):
        self.display = textDisplay.NullGraphics()
        self.ghosts = [ghostAgents.RandomGhost(i+1) for i in range(self.layout.getNumGhosts())]
        self.agents = [self.pacman] + self.ghosts
        self.game = self.rules.newGame(self.layout, self.pacman, self.ghosts,
                                       self.display, self.beQuiet, self.catchExceptions)

        # inform learning agents of the game start
        for i in range(len(self.agents)):
            agent = self.game.agents[i]
            if not agent:
                self.game.mute(i)
                # this is a null agent, meaning it failed to load
                # the other team wins
                print("Agent %d failed to load" % i, file=sys.stderr)
                self.game.unmute()
                self.game._agentCrash(i, quiet=True)
                return

        self.agentIndex = self.game.startingIndex
        self.numAgents = len(self.game.agents)

    def get_current_state(self):
        return self.game.state.deepCopy()

    def get_reward(self):
        return env.game.state.getScore()

    def update_game_state(self, action):
        # Execute the action
        self.game.state = self.game.state.generateSuccessor( self.agentIndex, action )
        # Change the display
        self.game.display.update( self.game.state.data )
        # Allow for game specific conditions (winning, losing, etc.)
        self.game.rules.process(self.game.state, self.game)
        # Track progress
        if self.agentIndex == self.numAgents + 1: self.game.numMoves += 1
        # Next agent
        self.agentIndex = ( self.agentIndex + 1 ) % self.numAgents

        if _BOINC_ENABLED:
            boinc.set_fraction_done(self.game.getProgress())

    def convert_state_to_image(self, state):
        state = str(state).split("\n")[:-2]
        new_state = np.zeros((self.layout.height, self.layout.width))
        state_dict = {
            '%': 0, '.': 225, 'o': 255,
            'G': 50, '<': 75, '>': 75,
            '^': 75, 'v': 75, ' ': 150,
            'P': 75
        }

        for i in range(self.layout.height):
            for j in range(self.layout.width):
                new_state[i][j] = state_dict[state[i][j]]

        new_state = new_state.reshape(-1)
        new_state /= 255.0

        return new_state

    def get_action_as_number(self, action):
        direction_to_action = {
            Directions.NORTH: 1,
            Directions.SOUTH: 2,
            Directions.EAST: 3,
            Directions.WEST: 4,
            Directions.STOP: 0
        }

        return direction_to_action[action]

    def step(self):
        initial_reward = self.get_reward()

        for agentIndex, agent in enumerate(self.game.agents):
            if not self.done():
                state = self.get_current_state()

                if agentIndex == 0:
                    state_as_image = self.convert_state_to_image(state)
                    legal = state.getLegalPacmanActions()
                    action = agent.getAction(state_as_image, legal)
                else:
                    action = agent.getAction(state)

                self.update_game_state(action)

                if agentIndex == 0:
                    state = state_as_image
                    action = self.get_action_as_number(action)
                    next_state = self.get_current_state()
                    next_state = self.convert_state_to_image(next_state)

                    reward = self.get_reward() - initial_reward
                    done = self.done()
                    agent.step(state, action, reward, next_state, done)

    def done(self, fast_check=False):
        if not self.game.gameOver:
            return False
        else:
            if fast_check: self.game.display.finish()
            return True

In [3]:
env = Environment()

In [4]:
n_episodes = 300
print_every = 10
wins = []
scores = []
for i_episode in range(1, n_episodes + 1):
    if i_episode % print_every == 0:
        print("Episode: ", i_episode)

    env.reset()

    while not env.done():
        env.step()

    scores.append(env.game.state.getScore())
    wins.append(env.game.state.isWin())
    winRate = wins.count(True)/ float(len(wins))
    if i_episode % print_every == 0:
        print("Episode: ", i_episode, end=" ")
        print('Average Score: {:.2f}'.format(sum(scores) / float(len(scores))))
        #print('Scores:       ', ', '.join([str(score) for score in scores]))
        #print('Win Rate:      %d/%d (%.2f)' % (wins.count(True), len(wins), winRate))
        #print('Record:       ', ', '.join([ ['Loss', 'Win'][int(w)] for w in wins]))

        print()

Episode:  10
Episode:  10 Average Score: -536.00

Episode:  20
Episode:  20 Average Score: -544.25

Episode:  30
Episode:  30 Average Score: -535.37

Episode:  40
Episode:  40 Average Score: -542.65

Episode:  50
Episode:  50 Average Score: -536.94

Episode:  60
Episode:  60 Average Score: -537.20

Episode:  70
Episode:  70 Average Score: -540.17

Episode:  80
Episode:  80 Average Score: -538.48

Episode:  90
Episode:  90 Average Score: -536.56

Episode:  100
Episode:  100 Average Score: -535.97

Episode:  110
Episode:  110 Average Score: -536.63

Episode:  120
Episode:  120 Average Score: -537.17

Episode:  130
Episode:  130 Average Score: -536.15

Episode:  140
Episode:  140 Average Score: -535.48

Episode:  150
Episode:  150 Average Score: -535.50

Episode:  160
Episode:  160 Average Score: -536.21

Episode:  170
Episode:  170 Average Score: -536.45

Episode:  180
Episode:  180 Average Score: -538.19

Episode:  190
Episode:  190 Average Score: -538.79

Episode:  200
Episode:  200 Av